In [13]:
# Bag-of-words using gensim.
from gensim.corpora.dictionary import Dictionary
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pandas as pd
from bs4 import BeautifulSoup
import pickle

path = '/Users/nemo/hku/mfin7036/nlp-ferrari'

In [3]:
#f = open("C:/Users/34433/Desktop/MFFT/Courses/MFIN7036 NLP/pure_articles/AAPL/2011-01-18T23_05_21-05_00_Apple Management Discusses Q1 2011 Results - Earnings Call Transcript.html",'r',encoding='utf-8')
def parse_html(file_path):
    f = open(file_path,'r',encoding='utf-8')
    htmlhandle=f.read()
    s= BeautifulSoup(htmlhandle,'lxml')
    text_list = [text for text in s.stripped_strings]
    return text_list
'''
for i in range(0,len(text_list)):
    print (text_list[i])
'''

'\nfor i in range(0,len(text_list)):\n    print (text_list[i])\n'

In [4]:
def get_mgmt_names(raw):
    hint_mgmt = ['Executives','Company Participants','Unverified Participant','Company Representatives','Corporate Participants']
    hint_analyst = ['Analysts','Conference Call Participants']
    hint_operator = ['Unidentified Company Representative','Operator']
    mgmt_index_flag = raw.index([x for x in hint_mgmt if x in raw ][0])
    # Some earnings call are without analysts
    try:
        analyst_index_flag = raw.index([x for x in hint_analyst if x in raw ][0])
    except:
        try: # Some transcripts may not have Analysts
            analyst_index_flag = raw.index("Operator")
        except: # Some transcripts may even not have Operator
            for t in raw:
                if len(t.split(" "))>35:
                    analyst_index_flag = raw.index(t)-1 # "-1"To make follwing analyst names blank
                    break
    # Deal with transcripts that do not start from Operator
    try:
        operator_index_flag = raw.index([x for x in hint_operator if x in raw ][0])
    except:
        for t in raw:
            if len(t.split(" "))>35:
                operator_index_flag = raw.index(t)
                break
    if operator_index_flag >25:
        for t in raw:
            if len(t.split(" "))>35:
                operator_index_flag = raw.index(t)
                break
    #print ("THE OPERATOR FLAG: ",operator_index_flag )
    if set(raw[mgmt_index_flag+1:analyst_index_flag]) & set (raw[operator_index_flag:]) == set():
        mgmt_names = [name.split(' - ')[0].split(' – ')[0] for name in raw[mgmt_index_flag+1:analyst_index_flag]]
        try:
            if analyst_index_flag != raw.index("Operator"): 
                # Set this condition to make sure analyst exist
                analyst_names = [name.split(' - ')[0].split(' – ')[0] for name in raw[analyst_index_flag+1:operator_index_flag]]                
            else:
                analyst_names = []
        except:
            analyst_names = []
    else:
        #print ("??????? THE SET IS ????", set(raw[mgmt_index_flag+1:analyst_index_flag]) & set (raw[operator_index_flag:]))
        mgmt_names = raw[mgmt_index_flag+1:analyst_index_flag]
        analyst_names = raw[analyst_index_flag+1:operator_index_flag]
    return mgmt_names, analyst_names
'''
    if 'Executives' in raw:
        if "Analysts" in raw:
            mgmt_names = [name.split(' - ')[0].split(' – ')[0] for name in raw[6:raw.index('Analysts')]]
            analyst_names = [name.split(' - ')[0].split(' – ')[0] for name in raw[raw.index('Analysts')+1:raw.index('Operator')]]
        elif 'Conference Call Participants' in raw:
            mgmt_names = [name.split(' - ')[0].split(' – ')[0] for name in raw[6:raw.index('Conference Call Participants')]]
            analyst_names = [name.split(' - ')[0].split(' – ')[0] for name in raw[raw.index('Conference Call Participants')+1:raw.index('Operator')]]
    elif 'Company Participants' in raw:
    elif 'Unverified Participant' in raw:
'''
    

'\n    if \'Executives\' in raw:\n        if "Analysts" in raw:\n            mgmt_names = [name.split(\' - \')[0].split(\' – \')[0] for name in raw[6:raw.index(\'Analysts\')]]\n            analyst_names = [name.split(\' - \')[0].split(\' – \')[0] for name in raw[raw.index(\'Analysts\')+1:raw.index(\'Operator\')]]\n        elif \'Conference Call Participants\' in raw:\n            mgmt_names = [name.split(\' - \')[0].split(\' – \')[0] for name in raw[6:raw.index(\'Conference Call Participants\')]]\n            analyst_names = [name.split(\' - \')[0].split(\' – \')[0] for name in raw[raw.index(\'Conference Call Participants\')+1:raw.index(\'Operator\')]]\n    elif \'Company Participants\' in raw:\n    elif \'Unverified Participant\' in raw:\n'

In [5]:
def get_mgmt_discussion(text_list, mgmt_names,analyst_names):
    hints = mgmt_names+analyst_names+['Operator','Unidentified Company Representative']
    #print(hints)
    combined_strs = []
    for i in range(0,len(mgmt_names)):
        disc = []
        flag = -1
        for text in text_list:
            if text ==mgmt_names[i]  and flag == -1:
                flag=flag * -1
                #print ('Find the name')
            elif text not in hints and flag ==1:
                disc.append(text)
                #print("$$$$$$$$Append {}'s text".format(mgmt_names[i]))
            elif text in hints and flag ==1:
                flag=flag*-1
                #print ('End this paragraph')
            elif text not in hints and flag== -1:
                continue
        combined_str = ("\n\n".join(disc))
        #print (combined_str)
        #print ("------------------------------------------")
        combined_strs.append(combined_str)       
    #print ("********The length of STR list is {} ***********".format(len(combined_strs)))
    return ("\n\n").join(combined_strs)

In [9]:
articles = pd.read_csv(path + '/data/locations.csv')
articles = articles[articles['Children'] == False]
articles = articles[articles['Company'] != 'WMT']
articles = articles[articles['Company'] != 'GOOGL']
articles.dropna(inplace=True)

articles.sort_values(by=['Company', 'Date'], inplace=True)
articles.reset_index(drop=True, inplace=True)

In [11]:
def get_text(row):
    file_path = path + '/data' + row['Path']
    text_list=parse_html(file_path)
    # print ("###TEXT GET###")
    mgmt_names, analyst_names = get_mgmt_names(text_list)
    #print ("*** MGMT ***",mgmt_names , analyst_names)
    combined_strs = get_mgmt_discussion(text_list, mgmt_names,analyst_names)
    return combined_strs

#str_arr = articles.head(2).apply(get_text, axis=1)
str_arr = articles.apply(get_text, axis=1)

In [ ]:
pickle.dump(str_arr.to_list(), open('./data/articles_arr.p', 'wb'))

In [27]:
tokens = [word_tokenize(s.lower()) for s in str_arr]

In [28]:
no_stops = [[t for t in token if t not in stopwords.words('english') and t.isalnum() and len(t) > 1] for token in tokens]

In [31]:
pickle.dump(no_stops, open(path + '/data/no_stops_words.p', 'wb'))